In [151]:
from qiskit import IBMQ, pulse, assemble
import numpy as np
from qiskit.providers.aer.pulse.system_models.hamiltonian_model import HamiltonianModel
from qiskit.pulse import Schedule, Play
from qiskit.tools import job_monitor
from qutip import sigmax, tensor, identity, Qobj
from qutip.control.pulseoptim import optimize_pulse_unitary
from qutip.qip.device import Processor
from src.qutip_helper import convert_qutip_ham
import qutip.logging_utils as logging
import qutip

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [152]:
from src.schedule_helper import channel_finder
# subsystem_list=[0,1]
subsystem_list=[0]
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibmq_singapore')

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ibmqfactory.load_account:WARNING:2020-08-27 12:30:24,345: Credentials are already in use. The existing account in the session will be replaced.


\begin{align}
&  H_{rot} = H_0 + \frac{dR}{dt} + \frac{1}{2} \Omega \left[\cos{\phi} \sigma^x - \sin{\phi} \sigma^y \right]\\
& \frac{dR}{dt} = \omega \left[ \sigma^z_{12} + \sigma^z_{23} \right]
\end{align}

Once we have this equation for the hamiltonian in the rotated frame, we can actually plug it in.
the key thing that we also have to do is the conversion from cos and sin to x and isin.
so we pull out the e^itheta and etc. to get
\begin{equation}
 H_{rot} = H_0 + \frac{dR}{dt} + \frac{1}{2} \Omega \left[\cos{\phi} \sigma^x - \sin{\phi} \sigma^y \right]\\
\end{equation}

In [153]:
backend.configuration().hamiltonian['vars']['wq0'] = 0

num_qub=backend.configuration().n_qubits
lst = ['omegad' + str(i) for i in range(num_qub)]
for ele in lst:
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 0.5 * getattr(backend.configuration(), 'hamiltonian')['vars'][ele]
# freq_est = 4.97e9

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [154]:
print(backend.configuration().hamiltonian['h_latex'])

\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{19}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{11,16}(\sigma_{11}^{+}\sigma_{16}^{-}+\sigma_{11}^{-}\sigma_{16}^{+}) + J_{10,11}(\sigma_{10}^{+}\sigma_{11}^{-}+\sigma_{10}^{-}\sigma_{11}^{+}) + J_{7,12}(\sigma_{7}^{+}\sigma_{12}^{-}+\sigma_{7}^{-}\sigma_{12}^{+}) + J_{5,6}(\sigma_{5}^{+}\sigma_{6}^{-}+\sigma_{5}^{-}\sigma_{6}^{+}) \\ & + J_{8,9}(\sigma_{8}^{+}\sigma_{9}^{-}+\sigma_{8}^{-}\sigma_{9}^{+}) + J_{15,16}(\sigma_{15}^{+}\sigma_{16}^{-}+\sigma_{15}^{-}\sigma_{16}^{+}) + J_{1,6}(\sigma_{1}^{+}\sigma_{6}^{-}+\sigma_{1}^{-}\sigma_{6}^{+}) + J_{18,19}(\sigma_{18}^{+}\sigma_{19}^{-}+\sigma_{18}^{-}\sigma_{19}^{+}) \\ & + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) + J_{16,17}(\sigma_{16}^{+}\sigma_{17}^{-}+\sigma_{16}^{-}\sigma_{17}^{+}) + J_{6,7}(\sigma_{6}^{+}\sigma_{7}^{-}+\sigma_{6}^{-}\sigma_{7}^{+}) + J_{12,13}(\sigma

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{19}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{11,16}(\sigma_{11}^{+}\sigma_{16}^{-}+\sigma_{11}^{-}\sigma_{16}^{+}) + J_{10,11}(\sigma_{10}^{+}\sigma_{11}^{-}+\sigma_{10}^{-}\sigma_{11}^{+}) + J_{7,12}(\sigma_{7}^{+}\sigma_{12}^{-}+\sigma_{7}^{-}\sigma_{12}^{+}) + J_{5,6}(\sigma_{5}^{+}\sigma_{6}^{-}+\sigma_{5}^{-}\sigma_{6}^{+}) \\ & + J_{8,9}(\sigma_{8}^{+}\sigma_{9}^{-}+\sigma_{8}^{-}\sigma_{9}^{+}) + J_{15,16}(\sigma_{15}^{+}\sigma_{16}^{-}+\sigma_{15}^{-}\sigma_{16}^{+}) + J_{1,6}(\sigma_{1}^{+}\sigma_{6}^{-}+\sigma_{1}^{-}\sigma_{6}^{+}) + J_{18,19}(\sigma_{18}^{+}\sigma_{19}^{-}+\sigma_{18}^{-}\sigma_{19}^{+}) \\ & + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) + J_{16,17}(\sigma_{16}^{+}\sigma_{17}^{-}+\sigma_{16}^{-}\sigma_{17}^{+}) + J_{6,7}(\sigma_{6}^{+}\sigma_{7}^{-}+\sigma_{6}^{-}\sigma_{7}^{+}) + J_{12,13}(\sigma_{12}^{+}\sigma_{13}^{-}+\sigma_{12}^{-}\sigma_{13}^{+}) \\ & + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) + J_{9,14}(\sigma_{9}^{+}\sigma_{14}^{-}+\sigma_{9}^{-}\sigma_{14}^{+}) + J_{2,3}(\sigma_{2}^{+}\sigma_{3}^{-}+\sigma_{2}^{-}\sigma_{3}^{+}) + J_{11,12}(\sigma_{11}^{+}\sigma_{12}^{-}+\sigma_{11}^{-}\sigma_{12}^{+}) \\ & + J_{17,18}(\sigma_{17}^{+}\sigma_{18}^{-}+\sigma_{17}^{-}\sigma_{18}^{+}) + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{5,10}(\sigma_{5}^{+}\sigma_{10}^{-}+\sigma_{5}^{-}\sigma_{10}^{+}) + J_{13,14}(\sigma_{13}^{+}\sigma_{14}^{-}+\sigma_{13}^{-}\sigma_{14}^{+}) \\ & + J_{3,8}(\sigma_{3}^{+}\sigma_{8}^{-}+\sigma_{3}^{-}\sigma_{8}^{+}) + J_{13,18}(\sigma_{13}^{+}\sigma_{18}^{-}+\sigma_{13}^{-}\sigma_{18}^{+}) + J_{7,8}(\sigma_{7}^{+}\sigma_{8}^{-}+\sigma_{7}^{-}\sigma_{8}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{3}^{(1,6)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{4}^{(2,1)}(t)+U_{5}^{(2,3)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{7}^{(3,4)}(t)+U_{6}^{(3,2)}(t)+U_{8}^{(3,8)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{9}^{(4,3)}(t))\sigma_{4}^{X} + \Omega_{d,5}(U_{11}^{(5,10)}(t)+U_{10}^{(5,6)}(t))\sigma_{5}^{X} \\ & + \Omega_{d,6}(U_{14}^{(6,7)}(t)+U_{12}^{(6,1)}(t)+U_{13}^{(6,5)}(t))\sigma_{6}^{X} + \Omega_{d,7}(U_{15}^{(7,6)}(t)+U_{16}^{(7,8)}(t)+U_{17}^{(7,12)}(t))\sigma_{7}^{X} \\ & + \Omega_{d,8}(U_{19}^{(8,7)}(t)+U_{18}^{(8,3)}(t)+U_{20}^{(8,9)}(t))\sigma_{8}^{X} + \Omega_{d,9}(U_{21}^{(9,8)}(t)+U_{22}^{(9,14)}(t))\sigma_{9}^{X} \\ & + \Omega_{d,10}(U_{24}^{(10,11)}(t)+U_{23}^{(10,5)}(t))\sigma_{10}^{X} + \Omega_{d,11}(U_{25}^{(11,10)}(t)+U_{26}^{(11,12)}(t)+U_{27}^{(11,16)}(t))\sigma_{11}^{X} \\ & + \Omega_{d,12}(U_{28}^{(12,7)}(t)+U_{30}^{(12,13)}(t)+U_{29}^{(12,11)}(t))\sigma_{12}^{X} + \Omega_{d,13}(U_{31}^{(13,12)}(t)+U_{32}^{(13,14)}(t)+U_{33}^{(13,18)}(t))\sigma_{13}^{X} \\ & + \Omega_{d,14}(U_{34}^{(14,9)}(t)+U_{35}^{(14,13)}(t))\sigma_{14}^{X} + \Omega_{d,15}(U_{36}^{(15,16)}(t))\sigma_{15}^{X} \\ & + \Omega_{d,16}(U_{38}^{(16,15)}(t)+U_{39}^{(16,17)}(t)+U_{37}^{(16,11)}(t))\sigma_{16}^{X} + \Omega_{d,17}(U_{41}^{(17,18)}(t)+U_{40}^{(17,16)}(t))\sigma_{17}^{X} \\ & + \Omega_{d,18}(U_{42}^{(18,13)}(t)+U_{43}^{(18,17)}(t)+U_{44}^{(18,19)}(t))\sigma_{18}^{X} + \Omega_{d,19}(U_{45}^{(19,18)}(t))\sigma_{19}^{X} \\ \end{align}


\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{4}\left(\frac{\omega_{q,i}}{2}(\mathbb{I}-\sigma_i^{z})+\frac{\Delta_{i}}{2}(O_i^2-O_i)+\Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{2,3}(\sigma_{2}^{+}\sigma_{3}^{-}+\sigma_{2}^{-}\sigma_{3}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{3}^{(2,1)}(t)+U_{4}^{(2,3)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{6}^{(3,4)}(t)+U_{5}^{(3,2)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{7}^{(4,3)}(t))\sigma_{4}^{X} \\ \end{align}

In [155]:
HamiltonianModel.from_dict(backend.configuration().hamiltonian, subsystem_list=[0])._system

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(<qiskit.providers.aer.pulse.qutip_extra_lite.qobj.Qobj at 0x7feff798e6d0>,
  'wq0/2'),
 (<qiskit.providers.aer.pulse.qutip_extra_lite.qobj.Qobj at 0x7feff798e820>,
  'delta0/2'),
 (<qiskit.providers.aer.pulse.qutip_extra_lite.qobj.Qobj at 0x7feff798e910>,
  '-delta0/2'),
 (<qiskit.providers.aer.pulse.qutip_extra_lite.qobj.Qobj at 0x7feff82740d0>,
  'omegad1*U0'),
 (<qiskit.providers.aer.pulse.qutip_extra_lite.qobj.Qobj at 0x7feff82740d0>,
  'omegad0*D0')]

In [156]:
ham = convert_qutip_ham(backend, subsystem_list, True)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [157]:
ham['H_d']

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0

In [158]:
# sigmax = Qobj([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
# Qobj(ham['H_c']['D1'].full())

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [159]:
# tensor(identity(3),sigmax)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [160]:
# Qobj(ham['H_c']['D0'].full())
# ham['H_c'].pop('U2')
ham['H_c'].pop('U0')

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.         0.11501632 0.        ]
 [0.11501632 0.         0.        ]
 [0.         0.         0.11501632]]

In [161]:
if 'U2' in ham['H_c'].keys():
    ham['H_c'].pop('U2')

ham['H_c'].keys()

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['D0'])

In [162]:
drift_hamiltonian = ham['H_d']

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [163]:
control_hamiltonians = [Qobj(control.full()) for control in (ham['H_c'].values())]

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [164]:
new_x = Qobj([[0,1,0], [1,0,0], [0,0,1]])
if len(subsystem_list) > 1:
    U_targ = tensor(new_x, identity(3))
    U_0 = tensor(identity(3), identity(3))
else:
    U_targ = new_x
    U_0 = identity(3)
# U_targ = tensor(identity(3), new_x)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [165]:
drift_hamiltonian = drift_hamiltonian# 0.0000001
dt=backend.configuration().dt
drift_hamiltonian = drift_hamiltonian
# dt = dt * 1e9

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


let's try a new hamiltonian, the one from the three level system, for starters ignoring
the 1-2 transition and just doing the 0-1 (for the transformation matrix R)

\begin{align}
& H_c = \frac{1}{2} \Omega
\end{align}

\begin{align}
    & H_{rot} = H_0 + \frac{dR}{dt} + \frac{1}{2} \Omega \left[ \cos{\phi} \sigma^x - \sin{\phi} \sigma^y \right] \\
    & H_c = \frac{1}{2} \Omega \left[ \sigma^x_{12}, i \sigma^y_{12} \right] \\
\end{align}

In [166]:
fac2 = 2
Hc1 = ham['H_c']['D0'] / fac2
Hc2 = ham['iH_c']['D0']/ fac2

if drift_hamiltonian == 0:
    drift_hamiltonian = Hc1 * 0
# drift_hamiltonian

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [167]:
control_hamiltonians = [Hc1, Hc2]

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [168]:
# n_ts = 1280
n_ts = 512
# n_ts = 320
# evo_time = np.pi * 6
evo_time = n_ts * dt * 1e9

fid_err_targ=1e-10
max_iter=200
max_wall_time=120
min_grad=1e-20
p_type='RND'
alg='GRAPE'

log_level = logging.INFO

result = optimize_pulse_unitary(drift_hamiltonian, control_hamiltonians, U_0, U_targ, n_ts,
                                evo_time,
                                fid_err_targ=fid_err_targ, min_grad=min_grad,
                                max_iter=max_iter, max_wall_time=max_wall_time,
                                out_file_ext=None, init_pulse_type=p_type,
                                log_level=log_level, gen_stats=True, alg=alg,
                                amp_lbound=-0.7, amp_ubound=0.7, phase_option='PSU')
pulse_seq = result.final_amps

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:qutip.control.dynamics:Setting memory optimisations for level 0
INFO:qutip.control.dynamics:Internal operator data type choosen to be <class 'numpy.ndarray'>
INFO:qutip.control.dynamics:phased dynamics generator caching True
INFO:qutip.control.dynamics:propagator gradient caching True
INFO:qutip.control.dynamics:eigenvector adjoint caching True
INFO:qutip.control.dynamics:use sparse eigen decomp False
INFO:qutip.control.pulseoptim:System configuration:
Drift Hamiltonian:
Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Control 1 Ham

In [169]:
# working_amps = result.initial_amps

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [170]:
# control_hamiltonians = control_hamiltonians[0]
# pulse_seq

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [171]:
# result.evo_full_final
result.evo_full_final.tidyup(1e-4)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+1.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j]]

In [172]:
channels = ham['H_c'].keys()
channels
# pulse_seq_dict = {val: [pulse_seq[i][j] for i in range(len(pulse_seq))] for j, val in enumerate(channels)}

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['D0'])

In [173]:
from src.helper import comp_converter
pulse_seq_dict = {'D0': comp_converter(0,1, pulse_seq)}

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [174]:
from qiskit.quantum_info import Statevector
state = Statevector([1,0,0,0,0,0,0,0,0])
state = Statevector([1,0,0])
operator = result.evo_full_final.full()
evolved =state.evolve(operator)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [175]:
from src.schedule_helper import sequence_converter
my_schedule = sequence_converter(pulse_seq_dict)


from qiskit.scheduler import measure_all
# my_schedule += measure([0, 0], backend) << my_schedule.duration
my_schedule += measure_all(backend) << my_schedule.duration

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/qiskit/pulse/library/sample_pulse.py:42: DeprecationWarning: SamplePulse has been renamed to Waveform and is deprecated. Please replace SamplePulse(samples, channel) with Waveform(samples, channel).
  warnings.warn("SamplePulse has been renamed to Waveform and is deprecated. " +


In [176]:
# my_schedule

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [177]:
num_shots =4096

grape_qobj = assemble(my_schedule,
                      backend=backend,
                      qubit_lo_freq=backend.defaults().qubit_freq_est,
                      meas_level=2,
                      meas_return='single',
                      shots=num_shots)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [178]:
job = backend.run(grape_qobj)
job_monitor(job)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Job Status: job has successfully run


In [179]:
from src.helper import qubit_distribution, comp_converter
counts = job.result().get_counts()
dist = qubit_distribution(counts)

print(dist)

/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{0: {0: 3873, 1: 223}, 1: {0: 3859, 1: 237}, 2: {0: 4069, 1: 27}, 3: {0: 4057, 1: 39}, 4: {0: 4056, 1: 40}, 5: {0: 4045, 1: 51}, 6: {0: 3952, 1: 144}, 7: {0: 4062, 1: 34}, 8: {0: 4038, 1: 58}, 9: {0: 4023, 1: 73}, 10: {0: 4073, 1: 23}, 11: {0: 4046, 1: 50}, 12: {0: 4038, 1: 58}, 13: {0: 3961, 1: 135}, 14: {0: 3846, 1: 250}, 15: {0: 4059, 1: 37}, 16: {0: 4048, 1: 48}, 17: {0: 3944, 1: 152}, 18: {0: 3941, 1: 155}, 19: {0: 542, 1: 3554}}


In [180]:
num=num_qub - 1
zeros = dist[num][0] / num_shots
ones = dist[num][1] / num_shots
print('distribution: 0: {}, 1: {}'.format(zeros, ones))
print('qutip guess: {}'.format(evolved.probabilities()))


distribution: 0: 0.13232421875, 1: 0.86767578125
qutip guess: [0. 1. 0.]


/Users/benrosand/anaconda3/envs/qoc/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
